In [ ]:
# Parameters
mlflow_tracking_uri = "http://127.0.0.1:5000"
experiment_name = "diabetes-expt"
train_size = 0.8
fix_imbalance = True
fold_shuffle = True
fold = 3
numeric_imputation = "median"
include_models = ["rf", "lightgbm", "lr"]
n_select = 1
sort = "AUC"
upstream = {"clean": {"nb": "D:\\Data Science\\ml-pipeline-config\\diabetes_ml\\products\\clean_train_data.ipynb", "data": "D:\\Data Science\\ml-pipeline-config\\diabetes_ml\\products\\cleaned_train_data.csv"}}
product = {"nb": "D:\\Data Science\\ml-pipeline-config\\diabetes_ml\\products\\train_model.ipynb", "config": "D:\\Data Science\\ml-pipeline-config\\diabetes_ml\\products\\config.pkl"}


In [ ]:
# Add description here
#

In [ ]:
# Uncomment the next two lines to enable auto reloading for imported modules
# %load_ext autoreload
# %autoreload 2
# For more info, see:
# https://docs.ploomber.io/en/latest/user-guide/faq_index.html#auto-reloading-code-in-jupyter

In [4]:
from pycaret.classification import *
import pandas as pd
import mlflow
# set the mlflow tracking uri
mlflow.set_tracking_uri(mlflow_tracking_uri)
# read the cleaned data
df = pd.read_csv(upstream['clean']['data'])
# setup pycaret
numeric_features = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']
clf = setup(data=df,
            target='Outcome',
            train_size = train_size,
            fold_shuffle = fold_shuffle,
            fold = fold,
            html = False,
            silent = True,
            numeric_features = numeric_features,
            experiment_name = experiment_name,
            log_experiment = True,
            fix_imbalance = fix_imbalance,
            numeric_imputation = numeric_imputation)
# save setup configuration
save_config(product['config'])
mlflow.log_artifact(product['config'])

best = compare_models(include_models)

final_model = finalize_model(tune_model(best, choose_better = True, optimize = 'AUC'))